<a href="https://colab.research.google.com/github/AyA-EhaB/FEDIS_Tasks/blob/main/CV_ResNet_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- AlexNet (2012): First breakthrough CNN. Shallow (8 layers), fast, weaker accuracy by today’s standards.

- ResNet152 (2015): Very deep (152 layers), uses skip connections so it can train without vanishing gradients. Much more accurate, but heavier and slower.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import classification_report, confusion_matrix

# Preprocessing (Transforms)
### Pretrained models expect images:
##### - size: 224x224
##### - normalized using ImageNet's mean & std

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

**'transforms.ToTensor()'**

What does it do?
- PyTorch expects images as tensors in a specific format:

- Channels first → (C, H, W)

- C = channels (3 for RGB), H = height, W = width

- But most image libraries (like PIL, OpenCV, NumPy) store them as:

- Channels last → (H, W, C)

- 👉 transforms.ToTensor() changes the order from (H, W, C) to (C, H, W).

In [ ]:
trainset = torchvision.datasets.CIFAR10(
    root = './data', train = True, download = True,
    transform = transform
)

100%|██████████| 170M/170M [00:01<00:00, 106MB/s]  


In [ ]:
trainloader  =torch.utils.data.DataLoader(trainset, batch_size = 32,shuffle = True)

In [ ]:
testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True,
    transform = transform )

In [ ]:
testloader  =torch.utils.data.DataLoader(testset, batch_size = 32,shuffle = False)

In [ ]:
classes = trainset.classes
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


## Load Models

In [ ]:
import torchvision.models as models
# ResNet152
resnet = models.resnet152(pretrained = True)
for param in resnet.parameters():
    param.requires_grad = False
resnet.fc = nn.Linear(resnet.fc.in_features, len(classes))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 210MB/s]  


In [ ]:
import torchvision.models as models
alexnet = models.alexnet(pretrained = True)
for param in alexnet.parameters():
    param.require_grad = False
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, len(classes))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 215MB/s]  


- param.requires_grad = False

In PyTorch, every parameter (weights & biases) has a flag requires_grad.

If True → gradients are computed during backpropagation (so it can be updated).

If False → parameter is frozen (not updated during training).

👉 Why do this?

We are doing transfer learning.

The pretrained AlexNet already knows useful features (edges, shapes, textures) from ImageNet.

We don’t want to “retrain” those millions of parameters (takes too long, needs huge data).

So we freeze them by setting requires_grad = False.

In [ ]:
# ==============================
# 5. Training
# ==============================
# What happens in each epoch:
# - Set model to training mode
# - Loop through batches
# - Forward pass → get predictions
# - Compute loss (CrossEntropy)
# - Backward pass → update weights
# - Print average loss per epoch

In [ ]:
def train(model, trainlaoder, epochs = 5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = 0.001)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images , labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss +=loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainloader):.4f}")
    return model

In [ ]:
def evaluate_model(model, testloader, classes):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():   # Disable gradient calculation
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)       # Forward pass
            _, preds = torch.max(outputs, 1)  # Get class with max probability
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=classes))

    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:\n", cm)


In [ ]:
print("\nTraining ResNet152...")
resnet_trained = train(resnet, trainloader, epochs=5)
print("\nEvaluating ResNet152...")
evaluate_model(resnet_trained, testloader, classes)

# Train & Evaluate AlexNet
print("\nTraining AlexNet...")
alexnet_trained = train(alexnet, trainloader, epochs=5)
print("\nEvaluating AlexNet...")
evaluate_model(alexnet_trained, testloader, classes)


Training ResNet152...
Epoch [1/5], Loss: 0.6833
Epoch [2/5], Loss: 0.5502
Epoch [3/5], Loss: 0.5326
Epoch [4/5], Loss: 0.5128
Epoch [5/5], Loss: 0.5024

Evaluating ResNet152...
Classification Report:
              precision    recall  f1-score   support

    airplane       0.82      0.88      0.85      1000
  automobile       0.89      0.91      0.90      1000
        bird       0.90      0.73      0.80      1000
         cat       0.74      0.73      0.74      1000
        deer       0.74      0.90      0.81      1000
         dog       0.78      0.85      0.82      1000
        frog       0.93      0.88      0.91      1000
       horse       0.91      0.82      0.86      1000
        ship       0.89      0.89      0.89      1000
       truck       0.92      0.87      0.90      1000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

Confusion Matrix:
 [[882  13   8  10  15